# Imports

In [28]:
import numpy as np 
import pandas as pd

## Getting the Wikipedia page

In [29]:
link = "http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

toronto_data = pd.read_html(link,header=0)[0]

toronto_data = toronto_data.replace(to_replace='Not assigned',value=np.nan)
toronto_data = toronto_data.loc[toronto_data.Borough.notna(),:]
toronto_data = toronto_data.apply(lambda x: [x.PostalCode,x.Borough, x.Borough] if pd.isna(x.Neighborhood)else x, axis=1)

#Renaming columns cause space char for Postal Code; substituting space with empty string:
cols = toronto_data.columns
cols = cols.map(lambda x: x.replace(' ', '') if isinstance(x, (str)) else x)
toronto_data.columns = cols

toronto_data.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Okay let's breakdown what I did here. I replaces Not assigned with a nan value and then dropped for boroughs and filled the neighboorhood names.

Next, I will conbine neighboorhoods with the same postal code as required

In [30]:
#counts = toronto_data.PostCode.value_counts()
counts = toronto_data.Postalcode.value_counts()
counts[counts>1]

Series([], Name: Postalcode, dtype: int64)

We need to combine these neighboorhood names separated by commas 

In [31]:
toronto_data = toronto_data.groupby('Postalcode').apply(lambda x: x.apply(lambda y:  ', '.join(y) \
                                                   if y.name =='Neighborhood' \
                                                   else y.tolist()[0]))
toronto_data.reset_index(inplace=True, drop=True)
toronto_data.loc[toronto_data.Postalcode.isin(counts[counts>1].index),:].head()

,Postalcode,Borough,Neighborhood


In [32]:
toronto_data.loc[toronto_data.Postalcode.isin(counts[counts==1].index),:].head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


We demonstrated that our code worked well

In [33]:
# The shape of the table 
toronto_data.shape

(103, 3)

In [34]:
toronto_data.to_csv('toronto_data.csv', index=False)